In [10]:
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # グラフ表示のために追加



st.title("家計簿分析アプリ")
st.write("毎月の支出データを入力して，傾向を分析しましょう．")

# 分析する項目を修正
items = ["食費", "住居費", "交通費", "通信費", "医療費", "水道光熱費"]
num_months = 12 # 1年分のデータなので12ヶ月に固定

st.subheader(f"過去{num_months}ヶ月の支出データを入力してください (単位: 円)")
st.write("各項目の金額を入力してください．")

# 支出データ入力用のDataFrameを作成
# 初期値としてゼロを設定
data = {f"{i+1}月": [0] * len(items) for i in range(num_months)}
df_expenses = pd.DataFrame(data, index=items)

# st.data_editorでユーザーに入力させる
edited_df = st.data_editor(
    df_expenses,
    column_config={
        f"{i+1}月": st.column_config.NumberColumn(
            label=f"{i+1}月",
            min_value=0,
            format="%.0f",
        ) for i in range(num_months)
    },
    num_rows="fixed", # 行の追加・削除は今回はなし
    height=400 # 表示の高さを調整
)

if st.button("支出を分析"):
    # pandas DataFrameをNumPy行列に変換
    expense_matrix = edited_df.values

    if expense_matrix.shape[1] > 0: # 月の数が0でないことを確認
        # 各項目の平均支出を計算 (行方向の平均)
        average_expenses = np.mean(expense_matrix, axis=1)

        st.subheader("分析結果: 各項目の平均支出")
        for i, item in enumerate(items):
            st.success(f"{item}: {average_expenses[i]:,.0f} 円")

        # 支出の合計も計算
        total_monthly_expenses = np.sum(expense_matrix, axis=0)
        st.subheader("月ごとの合計支出")
        for i, month_total in enumerate(total_monthly_expenses):
            st.info(f"{i+1}月の合計支出: {month_total:,.0f} 円")
        st.success(f"1年間の合計支出: {np.sum(total_monthly_expenses):,.0f} 円")


        st.subheader("線形代数の考え方")
        st.write("入力されたデータは，各行が支出項目，各列が月を表す行列として表現できます．")
        st.write("この行列の各行の平均を計算することで，項目ごとの平均支出を求めています．")
        st.write("また，行列の各列の合計を計算することで，月ごとの合計支出を求めています．")

        # 行列の表示を調整
        st.latex(r"\text{支出行列} = \begin{pmatrix}")
        for i, item in enumerate(items):
            row_str = " & ".join([f"{val:,.0f}" for val in expense_matrix[i]])
            st.latex(r"  " + row_str + r" \\")
        st.latex(r"\end{pmatrix}")
        st.latex(r"\text{平均支出ベクトル} = \text{mean(支出行列, axis=1)}")
        st.latex(r"\text{月別合計支出ベクトル} = \text{sum(支出行列, axis=0)}")

        # 支出の推移グラフ
        st.subheader("支出の月ごとの推移")
        fig, ax = plt.subplots(figsize=(12, 7)) # グラフサイズを少し大きく
        months = [f"{i+1}月" for i in range(num_months)]

        # 各項目の支出推移
        for i, item in enumerate(items):
            ax.plot(months, expense_matrix[i], marker='o', label=item)

        ax.set_xlabel("月")
        ax.set_ylabel("支出額 (円)")
        ax.set_title("各項目の支出推移")
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left') # 凡例を外に
        ax.grid(True)
        plt.xticks(rotation=45) # 月のラベルを斜めにする
        plt.tight_layout() # レイアウトを調整
        st.pyplot(fig) # Streamlitにグラフを表示

        # 月ごとの合計支出推移グラフ
        st.subheader("月ごとの合計支出推移")
        fig_total, ax_total = plt.subplots(figsize=(10, 6))
        ax_total.bar(months, total_monthly_expenses, color='skyblue')
        ax_total.set_xlabel("月")
        ax_total.set_ylabel("合計支出額 (円)")
        ax_total.set_title("月ごとの合計支出推移")
        ax_total.grid(axis='y')
        plt.xticks(rotation=45)
        plt.tight_layout()
        st.pyplot(fig_total)


    else:
        st.warning("分析する月のデータがありません．")

st.markdown("---")
st.write("### 発展的な機能のアイデア")
st.write("1. **予算との比較**: 各項目の予算額を入力する欄を設け，実績（入力データ）と比較して達成度を可視化する．")
st.write("2. **収入の追加**: 収入データも入力できるようにし，月ごとの収支を計算する．")
st.write("3. **目標設定**: 節約目標などを設定し，達成状況をトラッキングする．")
st.write("4. **データのエクスポート/インポート**: 入力したデータをCSVなどで保存・読み込みできるようにする．")

ModuleNotFoundError: No module named 'matplotlib'